In [1]:
import warnings
warnings.filterwarnings(action="ignore", category=DeprecationWarning)
warnings.filterwarnings(action="ignore", category=FutureWarning)
from LCWavelet import *
from tqdm import tqdm
from collections import defaultdict
from parallelbar import progress_map
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, concatenate,Conv1D, Flatten,Dropout , BatchNormalization, MaxPooling1D
from tensorflow.keras.models import Model, Sequential
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from functools import partial
import datetime

path = "all_data_2024-06-01/"
files = [file for file in os.listdir(path) if file.endswith(".pickle")]
lightcurves = []

def load_files(file, path):
    global_local = LightCurveWaveletGlobalLocalCollection.from_pickle(path+file)
    try:
        getattr(global_local, "levels")
    except AttributeError:
        global_local.levels = [1, 2, 3, 4]
    return global_local

func = partial(load_files, path=path)

# lightcurves = progress_map(func, files, n_cpu=128, total=len(files), executor='processes', error_behavior='raise')

for file in tqdm(files):
    lightcurves.append(func(file))

2024-06-11 20:24:53.329110: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9312/9312 [04:06<00:00, 37.83it/s]


In [2]:

pliegue_par_global = defaultdict(list)
pliegue_impar_global = defaultdict(list)
pliegue_par_local = defaultdict(list)
pliegue_impar_local = defaultdict(list)

for lc in lightcurves:
    for level in lc.levels:
        pliegue_par_global[level].append(lc.pliegue_par_global.get_approximation_coefficent(level=level))
        pliegue_impar_global[level].append(lc.pliegue_impar_global.get_approximation_coefficent(level=level))
        pliegue_par_local[level].append(lc.pliegue_par_local.get_approximation_coefficent(level=level))
        pliegue_impar_local[level].append(lc.pliegue_impar_local.get_approximation_coefficent(level=level))
        

pliegue_par_global = {k: np.array(v) for k, v in pliegue_par_global.items() if k in (1 ,2)}
pliegue_par_global = {k: v.reshape(list(v.shape)+[1]) for k, v in pliegue_par_global.items()}
pliegue_impar_global = {k: np.array(v) for k, v in pliegue_impar_global.items() if k in (1, 2)}
pliegue_impar_global = {k: v.reshape(list(np.shape(v))+[1]) for k, v in pliegue_impar_global.items()}


pliegue_par_local = {k: np.array(v) for k, v in pliegue_par_local.items() if k in (1, 2)}
pliegue_par_local = {k: v.reshape(list(np.shape(v))+[1]) for k, v in pliegue_par_local.items()}
pliegue_impar_local = {k: np.array(v) for k, v in pliegue_impar_local.items() if k in (1, 2)}
pliegue_impar_local = {k: v.reshape(list(np.shape(v))+[1]) for k, v in pliegue_impar_local.items()}


inputs = (pliegue_par_global, pliegue_impar_global), (pliegue_par_local, pliegue_impar_local)

In [3]:
# inputs
%pdb off
def gen_model_2_levels(inputs, classes, activation = 'relu',summary=False):
    
    (pliegue_par_global, pliegue_impar_global), (pliegue_par_local, pliegue_impar_local) = inputs    

    input_shape_global = [x.shape for x in pliegue_par_global.values()]
    assert input_shape_global == [x.shape for x in pliegue_impar_global.values()]
    
    input_shape_local = [x.shape for x in pliegue_par_local.values()]
    assert input_shape_local == [x.shape for x in pliegue_impar_local.values()]


    net = defaultdict(list)
 
    for (n, data), (n_inv, data_inv) in zip(sorted(pliegue_par_global.items(), key=lambda d: d[0]), sorted(pliegue_par_global.items(), key=lambda d: -d[0])):
        block = Sequential()
        for i in range(n_inv):
            block.add( Conv1D(16*2**n_inv, 5, activation=activation, input_shape=data.shape[1:], ))
            block.add(Conv1D(16*2**n_inv, 5, activation=activation, ))
            block.add(MaxPooling1D(pool_size=3, strides=1, ))

        block.add(Flatten())
        net["global_par"].append(block)
        
    for (n, data), (n_inv, data_inv) in zip(sorted(pliegue_impar_global.items(), key=lambda d: d[0]), sorted(pliegue_impar_global.items(), key=lambda d: -d[0])):
        block = Sequential()
        for i in range(n_inv):
            block.add(Conv1D(16*2**n_inv, 5, activation=activation, input_shape=data.shape[1:], ))
            block.add(Conv1D(16*2**n_inv, 5, activation=activation, ))
            block.add(MaxPooling1D(pool_size=3, strides=1, ))

        block.add(Flatten())
        net["global_impar"].append(block)

    for (n, data), (n_inv, data_inv) in zip(sorted(pliegue_par_local.items(), key=lambda d: d[0]), sorted(pliegue_par_local.items(), key=lambda d: -d[0])):
        block = Sequential()
        for i in range(n_inv):
            block.add(Conv1D(16*2**i, 5, activation=activation, input_shape=data.shape[1:], ))
            block.add(Conv1D(16*2**i, 5, activation=activation, ))
            block.add(MaxPooling1D(pool_size=3, strides=1, ))
        block.add(Flatten())
        net["local_par"].append(block)
        
    for (n, data), (n_inv, data_inv) in zip(sorted(pliegue_impar_local.items(), key=lambda d: d[0]), sorted(pliegue_impar_local.items(), key=lambda d: -d[0])):
        block = Sequential()
        for i in range(n_inv):
            block.add( Conv1D(16*2**i, 5, activation=activation, input_shape=data.shape[1:], ))
            block.add( Conv1D(16*2**i, 5, activation=activation, ))
            block.add(MaxPooling1D(pool_size=3, strides=1, ))
        block.add(Flatten())
        net["local_impar"].append(block)
                             
    model_f = concatenate([m.output for m in net["global_par"]] + [m.output for m in net["global_impar"]] + [m.output for m in net["local_par"]] + [m.output for m in net["local_impar"]], axis=-1)
    model_f = BatchNormalization(axis=-1)(model_f)
    model_f = Dense(512,activation=activation)(model_f)
    model_f = Dense(512,activation=activation)(model_f)
    model_f = Dense(512,activation=activation)(model_f)
    model_f = Dense(512,activation=activation)(model_f)
    model_f = Dense(len(classes),activation='softmax')(model_f)
    
    model_f = Model([[m.input for m in net["global_par"]], [m.input for m in net["global_impar"]]  , [m.input for m in net["local_par"]], [m.input for m in net["local_impar"]]],model_f)
    # model_f = Model([[model_p_7.input,model_i_7.input],[model_p_8.input,model_i_8.input]],model_f)


    if summary:
      model_f.summary()
    return model_f
    
    # model_f = concatenate([model_p_7.output,model_i_7.output,
    #                        model_p_8.output,model_i_8.output], axis=-1)
    # model_f = BatchNormalization(axis=-1)(model_f)
    # model_f = Dense(512,activation=activation)(model_f)
    # model_f = Dense(512,activation=activation)(model_f)
    # model_f = Dense(512,activation=activation)(model_f)
    # model_f = Dense(512,activation=activation)(model_f)
    # model_f = Dense(1,activation='sigmoid')(model_f)
    # model_f = Model([[model_p_7.input,model_i_7.input],[model_p_8.input,model_i_8.input]],model_f)

    # tamaño nivel 7
    input_shape_1 = np.shape(pliegue_par_global[1])[1:]
    # tamaño nivel 8
    input_shape_2 = np.shape(pliegue_par_global[2])[1:]
   
    # rama par level 7
    model_p_7 = tf.keras.Sequential()
    model_p_7.add(Conv1D(16, 5, activation=activation, input_shape=input_shape_1))
    model_p_7.add(Conv1D(16, 5, activation=activation)) 
    model_p_7.add(MaxPooling1D(pool_size=3, strides=1)) 
    model_p_7.add(Conv1D(32,5, activation=activation))
    model_p_7.add(Conv1D(32,5, activation=activation))
    model_p_7.add(MaxPooling1D(pool_size=3, strides=1))
    model_p_7.add(Conv1D(64,5, activation=activation))
    model_p_7.add(Conv1D(64,5, activation=activation))
    model_p_7.add(MaxPooling1D(pool_size=3, strides=1))
    model_p_7.add(Conv1D(128,5, activation=activation))
    model_p_7.add(Conv1D(128,5, activation=activation))
    model_p_7.add(MaxPooling1D(pool_size=3, strides=1))
    model_p_7.add(Flatten())
    
    # rama par level 8
    model_p_8 = tf.keras.Sequential()
    model_p_8.add(Conv1D(16, 5, activation=activation, input_shape=input_shape_2))
    model_p_8.add(Conv1D(16, 5, activation=activation)) 
    model_p_8.add(MaxPooling1D(pool_size=3, strides=1)) 
    model_p_8.add(Conv1D(32,5, activation=activation))
    model_p_8.add(Conv1D(32,5, activation=activation))
    model_p_8.add(MaxPooling1D(pool_size=3, strides=1))
    model_p_8.add(Conv1D(64,5, activation=activation))
    model_p_8.add(Conv1D(64,5, activation=activation))
    model_p_8.add(MaxPooling1D(pool_size=3, strides=1))
    model_p_8.add(Flatten())
    
    # rama impar level 7
    model_i_7 = tf.keras.Sequential()
    model_i_7.add(Conv1D(16, 5, activation=activation, input_shape=input_shape_1))
    model_i_7.add(Conv1D(16, 5, activation=activation)) 
    model_i_7.add(MaxPooling1D(pool_size=3, strides=1)) 
    model_i_7.add(Conv1D(32,5, activation=activation))
    model_i_7.add(Conv1D(32,5, activation=activation))
    model_i_7.add(MaxPooling1D(pool_size=3, strides=1))
    model_i_7.add(Conv1D(64,5, activation=activation))
    model_i_7.add(Conv1D(64,5, activation=activation))
    model_i_7.add(MaxPooling1D(pool_size=3, strides=1))
    model_i_7.add(Conv1D(128,5, activation=activation))
    model_i_7.add(Conv1D(128,5, activation=activation))
    model_i_7.add(MaxPooling1D(pool_size=3, strides=1))
    model_i_7.add(Flatten())
    
    # rama impar level 8
    model_i_8 = tf.keras.Sequential()
    model_i_8.add(Conv1D(16, 5, activation=activation, input_shape=input_shape_2))
    model_i_8.add(Conv1D(16, 5, activation=activation)) 
    model_i_8.add(MaxPooling1D(pool_size=3, strides=1)) 
    model_i_8.add(Conv1D(32,5, activation=activation))
    model_i_8.add(Conv1D(32,5, activation=activation))
    model_i_8.add(MaxPooling1D(pool_size=3, strides=1))
    model_i_8.add(Conv1D(64,5, activation=activation))
    model_i_8.add(Conv1D(64,5, activation=activation))
    model_i_8.add(MaxPooling1D(pool_size=3, strides=1))
    model_i_8.add(Flatten())

    # Red profunda
    model_f = concatenate([model_p_7.output,model_i_7.output,
                           model_p_8.output,model_i_8.output], axis=-1)
    
    import sys; sys.__breakpointhook__()
    model_f = BatchNormalization(axis=-1)(model_f)
    model_f = Dense(512,activation=activation)(model_f)
    model_f = Dense(512,activation=activation)(model_f)
    model_f = Dense(512,activation=activation)(model_f)
    model_f = Dense(512,activation=activation)(model_f)
    model_f = Dense(1,activation='sigmoid')(model_f)
    model_f = Model([[model_p_7.input,model_i_7.input],[model_p_8.input,model_i_8.input]],model_f)
    if summary:
      model_f.summary()
    return model_f



flatten = []
for (n, data) in sorted(pliegue_par_global.items(), key=lambda d: d[0]):
    flatten.append(data)
    
for (n, data) in sorted(pliegue_impar_global.items(), key=lambda d: d[0]):
    flatten.append(data)

for (n, data) in sorted(pliegue_impar_local.items(), key=lambda d: d[0]):
    flatten.append(data)
    
for (n, data) in sorted(pliegue_par_local.items(), key=lambda d: d[0]):
    flatten.append(data)

y = np.array([lc.headers['class'] for lc in lightcurves])
output_classes = np.unique([lc.headers['class'] for lc in lightcurves])
class2num = {label: n for n, label in enumerate(output_classes)}
num2class = {n: label for n, label in enumerate(output_classes)}

y = to_categorical([class2num[x] for x in y], num_classes=3)

res = train_test_split(*(flatten+[y]), test_size=0.3, shuffle=False)
*X_train, y_train = [r for n, r in enumerate(res) if n % 2 == 0 ]
*X_test, y_test = [r for n, r in enumerate(res) if n % 2 == 1 ]

model_1 = gen_model_2_levels(inputs, output_classes)
model_1.compile(loss = 'binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy','binary_crossentropy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=log_dir,
                                                 save_weights_only=True,
                                                 verbose=1)

history_1 = model_1.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test),
                        callbacks=[tensorboard_callback, cp_callback])

Automatic pdb calling has been turned OFF


2024-06-11 20:29:02.114904: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-11 20:29:02.146780: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-11 20:29:02.146911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-11 20:29:02.147550: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

Epoch 1/10


2024-06-11 20:29:05.205440: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2024-06-11 20:29:06.945519: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


102/102 [==============================] - ETA: 0s - loss: 0.7249 - accuracy: 0.5947 - binary_crossentropy: 0.7249
Epoch 1: saving model to logs/fit/20240611-202903
102/102 [==============================] - 17s 131ms/step - loss: 0.7249 - accuracy: 0.5947 - binary_crossentropy: 0.7249 - val_loss: 0.6470 - val_accuracy: 0.4324 - val_binary_crossentropy: 0.6470
Epoch 2/10
102/102 [==============================] - ETA: 0s - loss: 0.3736 - accuracy: 0.7507 - binary_crossentropy: 0.3736
Epoch 2: saving model to logs/fit/20240611-202903
102/102 [==============================] - 11s 111ms/step - loss: 0.3736 - accuracy: 0.7507 - binary_crossentropy: 0.3736 - val_loss: 0.6299 - val_accuracy: 0.5276 - val_binary_crossentropy: 0.6299
Epoch 3/10
102/102 [==============================] - ETA: 0s - loss: 0.1936 - accuracy: 0.8921 - binary_crossentropy: 0.1936
Epoch 3: saving model to logs/fit/20240611-202903
102/102 [==============================] - 12s 114ms/step - loss: 0.1936 - accuracy: 0.

TypeError: object of type 'NoneType' has no len()